# 📊 Evaluate VI→EN Model (While Training)

Download output từ kernel đang train VI→EN để evaluate.

---

In [ ]:
# 1. Setup Kaggle API credentials
# Get your API key from: https://www.kaggle.com/settings → API → Create New Token

import os
os.makedirs('/root/.config/kaggle', exist_ok=True)

# ========================================
# 👇 REPLACE with your credentials! 👇
# ========================================
KAGGLE_USERNAME = "YOUR_USERNAME"
KAGGLE_KEY = "YOUR_API_KEY"

with open('/root/.config/kaggle/kaggle.json', 'w') as f:
    f.write(f'{"username":"{KAGGLE_USERNAME}","key":"{KAGGLE_KEY}"}')

os.chmod('/root/.config/kaggle/kaggle.json', 0o600)
print("✓ Kaggle API configured")

In [ ]:
# 2. Download output từ kernel training VI→EN
# ========================================
# 👇 REPLACE with your kernel name! 👇
# ========================================
KERNEL_NAME = "your-username/your-vi-en-kernel-name"  # e.g., "kiengioisk/nlp-vi-en-train"

!kaggle kernels output {KERNEL_NAME} -p /kaggle/working/output_vi_en
print("\n✓ Download complete")

In [ ]:
# 3. Check downloaded files
!ls -la /kaggle/working/output_vi_en/
!echo "\n=== Model Output ==="
!ls -la /kaggle/working/output_vi_en/model_output_vi_en/ 2>/dev/null || echo "Looking for checkpoints_vi_en..."
!ls -la /kaggle/working/output_vi_en/checkpoints_vi_en/ 2>/dev/null

In [ ]:
# 4. Install deps & setup
!pip install -q sentencepiece sacrebleu python-dotenv tqdm pyyaml

# Clone repo for code
!rm -rf /kaggle/working/repo
!git clone https://github.com/TranKien2005/EV_Translate_Modle_NLP_Project.git /kaggle/working/repo
%cd /kaggle/working/repo

In [ ]:
# 5. Find checkpoint and tokenizer paths (VI→EN specific)
from pathlib import Path

output = Path('/kaggle/working/output_vi_en')

# Find files
ckpts = list(output.rglob('*.pt'))
toks = list(output.rglob('*.model'))

print("Checkpoints found:")
for c in ckpts:
    print(f"  {c}")

print("\nTokenizers found:")
for t in toks:
    print(f"  {t}")

# Set paths - VI→EN uses tokenizer_vi and tokenizer_en
CKPT = str(ckpts[0]) if ckpts else None

# Find VI (source) and EN (target) tokenizers
TOK_VI = None
TOK_EN = None
for t in toks:
    if 'vi' in t.name.lower():
        TOK_VI = str(t)
    elif 'en' in t.name.lower():
        TOK_EN = str(t)
    elif 'src' in t.name.lower():  # Fallback
        TOK_VI = str(t)
    elif 'tgt' in t.name.lower():  # Fallback
        TOK_EN = str(t)

print(f"\n📍 Checkpoint: {CKPT}")
print(f"📍 VI Tokenizer (src): {TOK_VI}")
print(f"📍 EN Tokenizer (tgt): {TOK_EN}")

In [ ]:
# 6. Load VI→EN model
from src.evaluate import load_translator

translator = load_translator(
    checkpoint_path=CKPT,
    vocab_src_path=TOK_VI,  # Vietnamese = source
    vocab_tgt_path=TOK_EN,  # English = target
    config_path='config/config_vi_en.yaml'
)
print("✓ VI→EN Model loaded!")

In [ ]:
# 7. Test Vietnamese → English translations
test_sentences = [
    "Xin chào",
    "Bạn khỏe không?",
    "Cảm ơn bạn",
    "Tôi yêu bạn",
    "Chào buổi sáng",
    "Hôm nay thời tiết rất đẹp",
    "Tôi thích học ngôn ngữ mới"
]

print("="*60)
print("🇻🇳 → 🇬🇧 Vietnamese to English Translation")
print("="*60)

for s in test_sentences:
    translation = translator.translate(s, beam_size=4)
    print(f"\n🔹 VI: {s}")
    print(f"🔹 EN: {translation}")

In [ ]:
# 8. Interactive translation (enter your own sentences)
print("Enter Vietnamese sentences to translate (type 'quit' to exit):")
print("-" * 50)

while True:
    try:
        vi_text = input("\n🇻🇳 VI: ")
        if vi_text.lower() == 'quit':
            break
        en_text = translator.translate(vi_text, beam_size=4)
        print(f"🇬🇧 EN: {en_text}")
    except:
        break